<img align="left" src="https://panoptes-uploads.zooniverse.org/project_avatar/86c23ca7-bbaa-4e84-8d8a-876819551431.png" type="image/png" height=100 width=100>
</img>
<h1 align="right">KSO Tutorials #6: Evaluate machine learning models</h1>
<h3 align="right">Written by @jannesgg and @vykanton</h3>
<h5 align="right">Last updated: Oct 14, 2022</h5>

# 1. Set up and requirements

### Install and import Python packages

In [14]:
from IPython.display import clear_output
try:
  import google.colab
  import os
  IN_COLAB = True
  print("Running in Colab...")

  # Clone repo
  !git clone --recurse-submodules https://github.com/ocean-data-factory-sweden/koster_yolov4.git
  !pip install -q --upgrade pip
  !pip install -qr koster_yolov4/requirements.txt
  !pip install -qr koster_yolov4/yolov5_tracker/requirements.txt
  
  # Fix libmagic issue
  !apt-get -qq update && apt-get -qq install -y libmagic-dev > /dev/null

  # Replace upsampling script with custom version
  os.chdir("koster_yolov4/tutorials")
  !mv ../src/upsampling.py /usr/local/lib/python3.7/dist-packages/torch/nn/modules/upsampling.py

  # Replace nearest neighbours script with custom version (due to relative path issue)
  !cp ../src/nn_matching.py ../yolov5_tracker/strong_sort/sort/nn_matching.py
  !cp ../src/track.py ../yolov5_tracker/track.py

  # Enable external widgets
  from google.colab import output
  output.enable_custom_widget_manager()

  # Ensure widgets are shown properly
  !jupyter nbextension enable --user --py widgetsnbextension
  !jupyter nbextension enable --user --py jupyter_bbox_widget

  print("All packages are installed and ready to go!")
  try:
    clear_output()
    print("All packages are installed and ready to go!")
  except:
    clear_output()
    print("There have been some issues installing the packages!")
except:
  IN_COLAB = False
  import sys
  import pkgutil
  if pkgutil.find_loader('torch') is None:
    !pip install -q --upgrade pip
    !pip install -q torch==1.8.0 torchvision==0.9.0
  # Replace nearest neighbours script with custom version (due to relative path issue)
  !cp ../src/nn_matching.py ../yolov5_tracker/strong_sort/sort/nn_matching.py
  !cp ../src/track.py ../yolov5_tracker/track.py
  # Ensure widgets are shown properly
  !jupyter nbextension enable --user --py widgetsnbextension
  !jupyter nbextension enable --user --py jupyter_bbox_widget
  clear_output()
  print("Running locally... you're good to go!")

Running locally... you're good to go!


In [15]:
# Set the directory of the libraries
import sys, os
from pathlib import Path
sys.path.append('..')

# Enables testing changes in utils
%load_ext autoreload
%autoreload 2

# Set to display dataframes as interactive tables
from itables import init_notebook_mode
init_notebook_mode(all_interactive=True)
from ipyfilechooser import FileChooser

# Import required modules
import kso_utils.tutorials_utils as t_utils
import kso_utils.t6_utils as t6

# Model-specific imports
import yolov5.train as train
import yolov5.val as val
import yolov5.detect as detect
import wandb

clear_output()
print("Packages loaded successfully")

Packages loaded successfully


# 2. Evaluate model on custom footage

### Choose your project

In [16]:
project_name = t_utils.choose_project()

Dropdown(description='Project:', options=('Koster_Seafloor_Obs', 'Spyfish_Aotearoa', 'SGU', 'Medins', 'Test ML…

### Choose model

In [17]:
model = t6.choose_model(project_name.value)

Dropdown(description='Select model:', layout=Layout(width='50%'), options=(('fathomnet-benthic-yolov5', 'fatho…

Output()

In [18]:
download_dir = t_utils.choose_folder(".", "where to download the model")

FileChooser(path='.', filename='', title='HTML(value='Choose location of where to download the model')', show_…

In [19]:
artifact_dir = t6.get_model(model_name = model.value,
                            project_name = project_name.value,
                            download_path = download_dir.selected)

INFO:root:Downloading model checkpoint...
wandb: Downloading large artifact fathomnet-benthic-yolov5:latest, 175.84MB. 3 files... Done. 0:0:0
INFO:root:Checkpoint downloaded.


### Choose custom footage

In [20]:
source = t_utils.choose_folder(".", "custom footage")

FileChooser(path='.', filename='', title='HTML(value='Choose location of custom footage')', show_hidden='False…

### Choose where to save runs (this should be left as default value in most cases)

In [21]:
save_dir = t_utils.choose_folder(".", "runs output")

FileChooser(path='.', filename='', title='HTML(value='Choose location of runs output')', show_hidden='False', …

In [23]:
run = wandb.init(entity='koster', project="model-evaluations")

wandb: wandb version 0.13.4 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade


### Run model over selected custom footage

In [34]:
conf_thres = t6.choose_conf()

FloatSlider(value=0.5, continuous_update=False, description='Confidence threshold:', max=1.0, readout_format='…

In [25]:
detect.run(weights=[f for f in Path(artifact_dir).iterdir() if f.is_file() and '.pt' in str(f)][0], 
           source=source.selected, conf_thres=conf_thres.value, save_txt=True, save_conf=True,
           project=save_dir.selected,
           name="detect")

INFO:yolov5:YOLOv5 🚀 2022-10-14 Python-3.8.8 torch-1.8.0 CPU

INFO:yolov5:Fusing layers... 
INFO:yolov5:Model summary: 476 layers, 91841704 parameters, 0 gradients
INFO:yolov5:video 1/1 (1/251) /Users/jurie.germishuys/Workspace/odf/koster-uw/models/yolov5/tutorials/test_vid/000203 TMBL-ROV 2000 Säcken EJ numrerade band_90.mp4: 512x640 (no detections), 6176.4ms
INFO:yolov5:video 1/1 (2/251) /Users/jurie.germishuys/Workspace/odf/koster-uw/models/yolov5/tutorials/test_vid/000203 TMBL-ROV 2000 Säcken EJ numrerade band_90.mp4: 512x640 (no detections), 5826.7ms
INFO:yolov5:video 1/1 (3/251) /Users/jurie.germishuys/Workspace/odf/koster-uw/models/yolov5/tutorials/test_vid/000203 TMBL-ROV 2000 Säcken EJ numrerade band_90.mp4: 512x640 (no detections), 6653.2ms
INFO:yolov5:video 1/1 (4/251) /Users/jurie.germishuys/Workspace/odf/koster-uw/models/yolov5/tutorials/test_vid/000203 TMBL-ROV 2000 Säcken EJ numrerade band_90.mp4: 512x640 (no detections), 6123.9ms
INFO:yolov5:video 1/1 (5/251) /Users

### View model output

In [26]:
eval_dir = t_utils.choose_folder(save_dir.selected if "save_dir" in vars() else '.', "runs output")

FileChooser(path='/Users/jurie.germishuys/Workspace/odf/koster-uw/models/yolov5/tutorials/wandb', filename='',…

In [ ]:
config = wandb.config
config.confidence_threshold = conf_thres.value
config.model_name = model.value
config.evaluation_directory = eval_dir.selected

In [27]:
my_data = wandb.Artifact("detection_output", type="raw_data")
my_data.add_dir(eval_dir.selected)
run.log_artifact(my_data)

wandb: Adding directory to artifact (/Users/jurie.germishuys/Workspace/odf/koster-uw/models/yolov5/tutorials/wandb/detect2)... Done. 0.0s


In [ ]:
t6.choose_files(eval_dir.selected)

### Investigate training and validation datasets

In [ ]:
train_dataset, val_dataset = t6.get_dataset(project_name.value, model.value)

#### Training set

In [ ]:
t6.get_data_viewer(os.path.join(train_dataset, "data/images"))

#### Validation set

In [ ]:
t6.get_data_viewer(os.path.join(val_dataset, "data/images"))

# 3. Track unique individuals (optional)

In [28]:
latest_tracker = t6.track_objects(source_dir=source.selected, 
                                  artifact_dir= artifact_dir,
                                  tracker_folder=eval_dir.selected,
                                  conf_thres=0.5,
                                  img_size=(720, 720),
                                  gpu=False)

INFO:yolov5:YOLOv5 🚀 2022-10-14 Python-3.8.8 torch-1.8.0 CPU

INFO:yolov5:Fusing layers... 
INFO:yolov5:Model summary: 476 layers, 91841704 parameters, 0 gradients
Downloading...
From: https://drive.google.com/uc?id=1Kkx2zW89jq_NETu4u42CFZTMVD5Hwm6e
To: /Users/jurie.germishuys/Workspace/odf/koster-uw/models/yolov5/tutorials/wandb/detect2/osnet_x0_25_msmt17.pt
100%|██████████| 9.34M/9.34M [00:00<00:00, 10.2MB/s]


Successfully loaded pretrained weights from "/Users/jurie.germishuys/Workspace/odf/koster-uw/models/yolov5/tutorials/wandb/detect2/osnet_x0_25_msmt17.pt"
** The following layers are discarded due to unmatched keys or layer size: ['classifier.weight', 'classifier.bias']


INFO:yolov5:No detections
INFO:yolov5:No detections
INFO:yolov5:No detections
INFO:yolov5:No detections
INFO:yolov5:No detections
INFO:yolov5:No detections
INFO:yolov5:No detections
INFO:yolov5:No detections
INFO:yolov5:No detections
INFO:yolov5:No detections
INFO:yolov5:No detections
INFO:yolov5:No detections
INFO:yolov5:No detections
INFO:yolov5:No detections
INFO:yolov5:No detections
INFO:yolov5:No detections
INFO:yolov5:No detections
INFO:yolov5:No detections
INFO:yolov5:No detections
INFO:yolov5:No detections
INFO:yolov5:No detections
INFO:yolov5:No detections
INFO:yolov5:No detections
INFO:yolov5:No detections
INFO:yolov5:No detections
INFO:yolov5:No detections
INFO:yolov5:No detections
INFO:yolov5:No detections
INFO:yolov5:No detections
INFO:yolov5:No detections
INFO:yolov5:No detections
INFO:yolov5:No detections
INFO:yolov5:No detections
INFO:yolov5:No detections
INFO:yolov5:No detections
INFO:yolov5:No detections
INFO:yolov5:No detections
INFO:yolov5:No detections
INFO:yolov5:

In [29]:
my_data = wandb.Artifact("tracker_output", type="raw_data")
my_data.add_dir(latest_tracker)
run.log_artifact(my_data)

wandb: Adding directory to artifact (/Users/jurie.germishuys/Workspace/odf/koster-uw/models/yolov5/tutorials/wandb/detect2/runs/track/exp/tracks)... Done. 0.0s


### Generate classification report and counts by species

In [30]:
csv_report = t6.generate_csv_report(eval_dir.selected)
wandb.log({"predictions": wandb.Table(dataframe=csv_report)})

INFO:root:Report created at /Users/jurie.germishuys/Workspace/odf/koster-uw/models/yolov5/tutorials/wandb/detect2/annotations.csv


In [31]:
tracking_report = t6.generate_counts(eval_dir.selected, latest_tracker, artifact_dir)
wandb.log({"tracking_counts": wandb.Table(dataframe=tracking_report)})

INFO:root:Report created at /Users/jurie.germishuys/Workspace/odf/koster-uw/models/yolov5/tutorials/wandb/detect2/annotations.csv
ERROR:root:No tracks found.
ERROR:root:No tracks to count.


🔴 <span style="color:red">&nbsp;NOTE: Run this cell to complete WANDB run, else artifacts will not be shown.

In [32]:
wandb.finish()

_runtime,3876
_timestamp,1666261194
_step,1


_runtime,▁█
_timestamp,▁█
_step,▁█


In [ ]:
# END 